# Subgraphs

Graphs such as
[StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html)'s
naturally can be composed. Creating subgraphs lets you build things like
[multi-agent teams](./multi_agent/hierarchical_agent_teams.ipynb), where each
team can track its own separate state.

You can add a `StateGraph` instance as a node by first
[compiling](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html#compile)
it to translate it to its lower-level Pregel operations.

The main thing you should note is ensuring the "handoff" from the calling graph
to the called graph behaves as expected.

Below are a couple of examples showing how to do so!

## Setup

First, install LangGraph.

```bash
yarn add langraph
```

Optionally, we can set API key for
[LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

In [ ]:
// Deno.env.set("OPENAI_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "ls__...");
// Deno.env.set("LANGCHAIN_CALLBACKS_BACKGROUND", "true");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_PROJECT", "Configuration: LangGraphJS");


## Create Parent + Child Graphs

For this example, we will create two graphs: a parent graph with a few nodes,
and a child graph that is added as a node in the parent.

For this example we will use the same `State` in both graphs, though we will
show how using the same keys can be a stumbling block if you're not careful.

In [8]:
import { END, START, StateGraph } from "@langchain/langgraph";

function reduceList(
  left?: string[] | string,
  right?: string[] | string,
): string[] {
  if (!left) {
    left = [];
  } else if (typeof left === "string") {
    left = [left];
  }
  if (!right) {
    right = [];
  } else if (typeof right === "string") {
    right = [right];
  }
  return [...left, ...right];
}

// Define the state type
interface IState {
  name: {
    value: (x: string, y?: string) => string;
    default: () => string;
  };
  path: {
    value: (x?: string[], y?: string[] | string) => string[];
    default: () => string[];
  };
}

const graphState: IState = {
  name: {
    // Overwrite name if a new one is provided
    value: (x: string, y?: string) => (y ? y : x),
    default: () => "default",
  },
  path: {
    // Concatenate paths
    value: reduceList,
    default: () => [],
  },
};

const childBuilder = new StateGraph({ channels: graphState });
childBuilder.addNode("child_start", (state) => ({ path: ["child_start"] }));
childBuilder.addEdge(START, "child_start");
childBuilder.addNode("child_middle", (state) => ({ path: ["child_middle"] }));
childBuilder.addNode("child_end", (state) => ({ path: ["child_end"] }));
childBuilder.addEdge("child_start", "child_middle");
childBuilder.addEdge("child_middle", "child_end");
childBuilder.addEdge("child_end", END);

const builder = new StateGraph({
  channels: graphState,
});

builder.addNode("grandparent", (state) => ({ path: ["grandparent"] }));
builder.addEdge(START, "grandparent");
builder.addNode("parent", (state) => ({ path: ["parent"] }));
builder.addNode("child", childBuilder.compile());
builder.addNode("sibling", (state) => ({ path: ["sibling"] }));
builder.addNode("fin", (state) => ({ path: ["fin"] }));

// Add connections
builder.addEdge("grandparent", "parent");
builder.addEdge("parent", "child");
builder.addEdge("parent", "sibling");
builder.addEdge("child", "fin");
builder.addEdge("sibling", "fin");
builder.addEdge("fin", END);
const graph = builder.compile();

const result1 = await graph.invoke({ name: "test" });
console.log(result1);

{
  name: "test",
  path: [
    "grandparent",
    "parent",
    "grandparent",
    "parent",
    "child_start",
    "child_middle",
    "child_end",
    "sibling",
    "fin"
  ]
}


Notice here that the `["grandparent", "parent"]` sequence is duplicated! This is
because our child state has received the full parent state and returns the full
parent state once it terminates. In the next section, we will show how you can merge or separate state within nested graphs.

## State handoff

To avoid duplication or conflicts in state, you
typically would do one or more of the following:

1. Handle duplicates in your `reducer` function.
2. Call the child graph from within a TypeScript function. In that function,
   handle the state as needed.
3. Update the child graph keys to avoid conflicts. You would still need to
   ensure the output can be interpreted by the parent, however.

Let's re-implement the graph using technique (1) and add unique IDs for every
value in the list.

In [9]:
import { v4 as uuidv4 } from "uuid";

type ValWithId = { id?: string; val: string };

function reduceListWithIds(
  left?: ValWithId[] | ValWithId,
  right?: ValWithId[] | ValWithId,
): any[] {
  /**
   * Append the right-hand list, replacing any elements with the same id in the left-hand list.
   */
  if (!left) {
    left = [];
  } else if (!Array.isArray(left)) {
    left = [left];
  }
  if (!right) {
    right = [];
  } else if (!Array.isArray(right)) {
    right = [right];
  }
  // Ensure there's an id for each element
  const [left_, right_] = [left, right].map((orig) =>
    orig.map((val) => {
      if (!val?.id) {
        val.id = uuidv4();
      }
      return val;
    })
  );

  // Merge the two lists
  const leftIdxById = left_.reduce(
    (acc, val, i) => ({ ...acc, [val.id as string]: i }),
    {} as Record<string, number>,
  );
  const merged = [...left_];
  for (const val of right_) {
    const existingIdx = leftIdxById[val.id as string];
    if (existingIdx !== undefined) {
      merged[existingIdx] = val;
    } else {
      merged.push(val);
    }
  }
  return merged;
}

interface IStateWithIds {
  name: {
    value: (x: string, y?: string) => string;
    default: () => string;
  };
  path: {
    value: (x?: ValWithId[], y?: ValWithId[] | ValWithId) => ValWithId[];
    default: () => ValWithId[];
  };
}

const graphState2: IStateWithIds = {
  name: {
    // Overwrite name if a new one is provided
    value: (x: string, y?: string) => (y ? y : x),
    default: () => "default",
  },
  path: {
    // Concatenate paths
    value: reduceListWithIds,
    default: () => [],
  },
};

const childBuilderWithIds = new StateGraph({ channels: graphState2 });

childBuilderWithIds.addNode("child_start", (state) => ({
  path: [{ val: "child_start" }],
}));
childBuilderWithIds.setEntryPoint("child_start");
childBuilderWithIds.addNode("child_middle", (state) => ({
  path: [{ val: "child_middle" }],
}));
childBuilderWithIds.addNode("child_end", (state) => ({
  path: [{ val: "child_end" }],
}));
childBuilderWithIds.addEdge("child_start", "child_middle");
childBuilderWithIds.addEdge("child_middle", "child_end");
childBuilderWithIds.setFinishPoint("child_end");

const builderWithIds = new StateGraph({
  channels: graphState2,
});

builderWithIds.addNode("grandparent", (state) => ({
  path: [{ val: "grandparent" }],
}));
builderWithIds.setEntryPoint("grandparent");
builderWithIds.addNode("parent", (state) => ({ path: [{ val: "parent" }] }));
builderWithIds.addNode("child", childBuilderWithIds.compile());
builderWithIds.addNode("sibling", (state) => ({ path: [{ val: "sibling" }] }));
builderWithIds.addNode("fin", (state) => ({ path: [{ val: "fin" }] }));

// Add connections
builderWithIds.addEdge("grandparent", "parent");
builderWithIds.addEdge("parent", "child");
builderWithIds.addEdge("parent", "sibling");
builderWithIds.addEdge("child", "fin");
builderWithIds.addEdge("sibling", "fin");
builderWithIds.setFinishPoint("fin");
const graphWithIds = builderWithIds.compile();

const result2 = await graphWithIds.invoke({ name: "test" });
console.log(result2);

{
  name: "test",
  path: [
    { val: "grandparent", id: "1f8694a6-daba-4acf-86c5-394ceff7d9f1" },
    { val: "parent", id: "7e9505db-be27-465c-bdf3-fca5981449ff" },
    { val: "child_start", id: "e0c66b89-13e4-441b-93a2-3e7925294688" },
    { val: "child_middle", id: "ceeaae44-1342-42c5-a670-c69601586433" },
    { val: "child_end", id: "a5b00b2d-abca-4d87-ac72-8c85c3b7aa24" },
    { val: "sibling", id: "4acec7d3-a573-4d9d-96c2-ebcad5f18227" },
    { val: "fin", id: "2f72a96c-1097-40a5-b19f-558dcf89d8fc" }
  ]
}


Duplicates are gone!